In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
pip install requests yfinance mplfinance matplotlib pandas numpy fpdf flask gunicorn fpdf2 nselib

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.3/236.3 kB 9.9 MB/s eta 0:00:00
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=aa77070cd2ffd93c21541c0b102264b464fb88a86c1877be4bb90b80e3fc183d
  Stored in directory: /root/.cache/pip/wheels/f9/95/ba/f418094659025eb9611f17cbcaf2334236bf39a0c3453ea455
Successfully built fpdf
Note: you may need to restart the kernel to use updated packages.


In [3]:
## best w cross over code
import matplotlib.pyplot as plt
import yfinance as yf
import pandas as pd
import mplfinance as mpf
from fpdf import FPDF
print("here")
from datetime import datetime, timedelta
import pytz
from nselib import capital_market
import os
# flask
from flask import Flask, jsonify
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
# ----------------------------
# Email and Flask Setup
# ----------------------------
user_emails = ["ai20.vivek.patel@gmail.com", "kachhiaaryan1@gmail.com"]

EMAIL_CONFIG = {
    'address': "ai20.vivek.patel@gmail.com",
    'password': "kqql idxz noog lxpt",
    'smtp_server': "smtp.gmail.com",
    'smtp_port': 587
}

def send_email(attachments, recipient_email):
    """Send email with multiple attachments"""
    msg = MIMEMultipart()
    msg['From'] = EMAIL_CONFIG['address']
    msg['To'] = recipient_email
    msg['Subject'] = "📈 Stock Crossover Analysis Report from Vivek Patel"

    # HTML body for the email
    body = """
    <html>
        <body style="font-family: Arial, sans-serif; line-height: 1.6; color: #333;">
            <h2 style="color: #4CAF50;">Hello, User</h2>
            <p>We are pleased to share with you the latest <strong>Stock Crossover Analysis Reports</strong>.</p>
            <p>Please find the attached reports below:</p>
            <ul>
                <li><strong>Bullish Crossovers</strong></li>
                <li><strong>Bearish Crossovers</strong></li>
                <li><strong>All Ticker Charts</strong></li>
            </ul>
            <p>If you have any questions or need further assistance, feel free to reach out!</p>
            <p>Best regards,<br>
            Vivek Patel<br>
            <a href="mailto:ai20.vivek.patel@gmail.com">ai20.vivek.patel@gmail.com</a></p>
        </body>
    </html>
    """
    msg.attach(MIMEText(body, 'html'))

    for file in attachments:
        with open(file, 'rb') as f:
            part = MIMEBase('application', 'octet-stream')
            part.set_payload(f.read())
            encoders.encode_base64(part)
            part.add_header('Content-Disposition',
                            f'attachment; filename="{os.path.basename(file)}"')
            msg.attach(part)

    with smtplib.SMTP(EMAIL_CONFIG['smtp_server'], EMAIL_CONFIG['smtp_port']) as server:
        server.starttls()
        server.login(EMAIL_CONFIG['address'], EMAIL_CONFIG['password'])
        server.sendmail(EMAIL_CONFIG['address'], recipient_email, msg.as_string())

# --------------------------
# Configuration
# --------------------------
class Config:
    EMA_PERIODS = [10, 50, 100]
    CROSSOVER_LOOKBACK = 5
    INSTITUTIONAL_FILTERS = {
        'min_mcap': 20000,
        'avg_volume': 500000
    }

# --------------------------
# Data Acquisition
# --------------------------
def get_52_week_high_tickers(max_attempts=3):
    """Fetch 52-week high stocks with error handling"""
    india_tz = pytz.timezone("Asia/Kolkata")
    print("here23")
    print(india_tz)
    for attempt in range(max_attempts):
        trade_date = (datetime.now(india_tz) - timedelta(days=attempt)).strftime("%d-%m-%Y")
        print("trade_date",trade_date)
        try:
            report = capital_market.week_52_high_low_report(trade_date)
            print("trade_date2",report)
            prev_date = datetime.now(india_tz) - timedelta(days=attempt+1)
            prev_date_str = prev_date.strftime("%d-%b-%Y").upper()
            print("trade_date2",trade_date)
            new_highs = report[report["52_Week_High_Date"] == prev_date_str]
            return new_highs["SYMBOL"].tolist()
        except Exception as e:
            print(f"Attempt {attempt+1} failed: {str(e)}")

    return []

# --------------------------
# Technical Analysis Engine
# --------------------------
class CrossoverAnalyzer:
    def __init__(self, symbol):
        self.symbol = symbol
        self.data = self._get_data()
        self.emas = {}
        self.crossovers = {}
    def calculate_emas(self):
        """Calculate EMAs with validation"""
        if not self.data.empty:
            for period in Config.EMA_PERIODS:
                self.emas[period] = self.data['Close'].ewm(span=period, adjust=False).mean()
            return True
        return False
    def _get_data(self):
        """Fetch data with proper length validation"""
        try:
            df = yf.Ticker(f"{self.symbol}.NS").history(period='1y', interval='1d')
            return df[['Open','High','Low','Close','Volume']].iloc[-250:]  # Limit to 250 days
        except Exception as e:
            print(f"Data error for {self.symbol}: {str(e)}")
            return pd.DataFrame()

    def detect_crossovers(self):
        """Fixed indexing implementation"""
        self.crossovers = {'bullish': [], 'bearish': []}
        pairs = [(10,50), (10,100), (50,100)]

        for short, long in pairs:
            if len(self.data) < Config.CROSSOVER_LOOKBACK:
                continue

            # Use iloc for positional indexing
            recent_short = self.emas[short].iloc[-Config.CROSSOVER_LOOKBACK:]
            recent_long = self.emas[long].iloc[-Config.CROSSOVER_LOOKBACK:]
            diff = recent_short - recent_long

            for i in range(1, len(diff)):
                if diff.iloc[i-1] < 0 and diff.iloc[i] > 0:
                    self.crossovers['bullish'].append(f"EMA{short}/EMA{long}")
                elif diff.iloc[i-1] > 0 and diff.iloc[i] < 0:
                    self.crossovers['bearish'].append(f"EMA{short}/EMA{long}")

        return self.crossovers

    # Fixed legend colors
    def plot_analysis(self):
      """Fixed plotting with aligned indices"""
      if self.data.empty or len(self.data) < 30:
          return None

      plot_days = min(120, len(self.data))  # Dynamic plot range
      plot_data = self.data.iloc[-plot_days:]

      apds = [
          mpf.make_addplot(self.emas[10].iloc[-plot_days:], color='blue'),
          mpf.make_addplot(self.emas[50].iloc[-plot_days:], color='orange'),
          mpf.make_addplot(self.emas[100].iloc[-plot_days:], color='green')
      ]

      fig, ax = mpf.plot(plot_data,
                        type='candle',
                        addplot=apds,
                        volume=True,
                        style='yahoo',
                        returnfig=True)

      ax[0].set_title(f"{self.symbol} - EMA Crossovers")
      ax[0].set_xlabel('Date')
      ax[0].set_ylabel('Price')
      ax[0].grid(True)

      # Explicitly define the legend handles with the correct colors
      ema10_line, = ax[0].plot([], [], color='blue', label='EMA10')  # Empty line for legend
      ema50_line, = ax[0].plot([], [], color='orange', label='EMA50')
      ema100_line, = ax[0].plot([], [], color='green', label='EMA100')

      # Manually set the legend
      ax[0].legend(handles=[ema10_line, ema50_line, ema100_line],
                  loc='upper left',
                  facecolor='white',  # Optional: set the legend background color
                  frameon=True,       # Optional: whether to display a frame
                  handlelength=2,     # Optional: length of the legend handles
                  title="Exponential Moving Averages")  # Optional: title for the legend
      # Annotate only visible area
      for i in range(len(plot_data)):
          idx = i + (len(self.data) - plot_days)
          for crossover in self.crossovers['bullish']:
              short, long = map(int, crossover.replace('EMA', '').split('/'))
              if (self.emas[short].iloc[idx] > self.emas[long].iloc[idx] and
                  self.emas[short].iloc[idx-1] <= self.emas[long].iloc[idx-1]):
                  ax[0].annotate('▲', (i, plot_data['Close'].iloc[i]),
                                color='g', fontsize=9)

          for crossover in self.crossovers['bearish']:
              short, long = map(int, crossover.replace('EMA', '').split('/'))
              if (self.emas[short].iloc[idx] < self.emas[long].iloc[idx] and
                  self.emas[short].iloc[idx-1] >= self.emas[long].iloc[idx-1]):
                  ax[0].annotate('▼', (i, plot_data['Close'].iloc[i]),
                                color='r', fontsize=9)

      chart_path = f"{self.symbol}_chart.png"
      fig.savefig(chart_path, dpi=100, bbox_inches='tight')
      plt.close()
      return chart_path



class InstitutionalReport(FPDF):
    def add_stock_analysis(self, symbol, crossovers, chart_path):
        # Add a new page for each stock analysis
        self.add_page()

        # Header
        self.set_font('Helvetica', 'B', 14)
        self.cell(0, 10, f'{symbol} Institutional Analysis', 0, 1)

        # Crossover Signals
        self.set_font('Helvetica', 'B', 12)
        self.cell(0, 8, 'EMA Crossovers:', 0, 1)
        self.set_font('Helvetica', '', 10)

        if crossovers['bullish']:
            self.set_text_color(0, 128, 0)
            self.cell(0, 8, f'Bullish: {", ".join(crossovers["bullish"])}', 0, 1)

        if crossovers['bearish']:
            self.set_text_color(255, 0, 0)
            self.cell(0, 8, f'Bearish: {", ".join(crossovers["bearish"])}', 0, 1)

        self.set_text_color(0, 0, 0)

        # Chart
        self.image(chart_path, x=10, w=190)
        os.remove(chart_path)

# --------------------------
# Execution Workflow
# --------------------------
def generate_full_report():
    report = InstitutionalReport()  # Create report instance once
    print("here2")
    # Get 52-week high stocks
    high_tickers = get_52_week_high_tickers()
    print(f"52-week high stocks found: {len(high_tickers)}")

    for symbol in high_tickers:
        print(f"Analyzing {symbol}...")
        analyzer = CrossoverAnalyzer(symbol)
        if not analyzer.data.empty:
            analyzer.calculate_emas()
            crossovers = analyzer.detect_crossovers()
            chart_path = analyzer.plot_analysis()

            if chart_path:
                report.add_stock_analysis(symbol, crossovers, chart_path)

    # Output the report only once after adding all stock analyses
    report.output('Institutional_Crossover_Report.pdf')
    for recipient in user_emails:
        send_email(['Institutional_Crossover_Report.pdf'], recipient)
    print("Professional report generated")


# --------------------------
# Main Execution
# --------------------------
app = Flask(__name__)
# @app.route('/generate_report', methods=['GET'])
def generate_report():
    generate_full_report()
    # return jsonify({"message": "Report generation initiated"})

# Requirements:
# pip install yfinance pandas mplfinance nselib fpdf2 scipy
if __name__ == '__main__':
    # app.run(host='0.0.0.0', port=5000)
    generate_report()

/usr/local/lib/python3.10/dist-packages/fpdf/__init__.py:40: UserWarning: You have both PyFPDF & fpdf2 installed. Both packages cannot be installed at the same time as they share the same module namespace. To only keep fpdf2, run: pip uninstall --yes pypdf && pip install --upgrade fpdf2
  warnings.warn(


here
here2
here23
Asia/Kolkata
trade_date 06-02-2025
Attempt 1 failed:  Data not found, change the trade_date...
trade_date 05-02-2025
trade_date2           SYMBOL SERIES Adjusted52_Week_High 52_Week_High_Date  \
0      20MICRONS     EQ               348.00       22-AUG-2024   
1     21STCENMGM     BE               141.40       11-SEP-2024   
2         360ONE     EQ              1318.00       06-JAN-2025   
3      3IINFOLTD     EQ                61.20       07-FEB-2024   
4        3MINDIA     EQ             40856.50       02-JUL-2024   
...          ...    ...                  ...               ...   
2468        ZOTA     EQ              1060.00       16-JAN-2025   
2469       ZUARI     EQ               268.00       03-DEC-2024   
2470    ZUARIIND     EQ               423.40       08-JUL-2024   
2471   ZYDUSLIFE     EQ              1324.30       09-AUG-2024   
2472   ZYDUSWELL     EQ              2484.00       30-JUL-2024   

     Adjusted52_Week_Low 52_Week_Low_DT  
0                 

<ipython-input-3-d9af0657d8d1>:225: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  self.cell(0, 10, f'{symbol} Institutional Analysis', 0, 1)
<ipython-input-3-d9af0657d8d1>:229: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  self.cell(0, 8, 'EMA Crossovers:', 0, 1)


Analyzing AXISGOLD...


<ipython-input-3-d9af0657d8d1>:225: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  self.cell(0, 10, f'{symbol} Institutional Analysis', 0, 1)
<ipython-input-3-d9af0657d8d1>:229: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  self.cell(0, 8, 'EMA Crossovers:', 0, 1)


...

Analyzing VMM...
Analyzing WEL...


<ipython-input-3-d9af0657d8d1>:225: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  self.cell(0, 10, f'{symbol} Institutional Analysis', 0, 1)
<ipython-input-3-d9af0657d8d1>:229: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  self.cell(0, 8, 'EMA Crossovers:', 0, 1)


Analyzing WOCKPHARMA...


<ipython-input-3-d9af0657d8d1>:225: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  self.cell(0, 10, f'{symbol} Institutional Analysis', 0, 1)
<ipython-input-3-d9af0657d8d1>:229: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  self.cell(0, 8, 'EMA Crossovers:', 0, 1)
<ipython-input-3-d9af0657d8d1>:234: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  self.cell(0, 8, f'Bullish: {", ".join(crossovers["bullish"])}', 0, 1)


Professional report generated
